## Day 8 - Advent of Code

### Part A

In [3]:
%%time 

import numpy as np
import heapq
import math

# read input file and split each line by comma
with open("input8.txt") as file:
    data = [line.strip().split(',') for line in file]

# convert each line from strings to a tuple of ints
data = [tuple(map(int, coords)) for coords in data]

length = range(len(data))

# compute distances between every pair of points
distances = {}
for i in range(len(data)):
    a = np.array(data[i], dtype=int)
    for j in range(i+1, len(data)):
        b = np.array(data[j], dtype=int)
        dist = float(np.linalg.norm(a - b))  # euclidean distance
        distances[(i, j)] = {
            "a": a.tolist(),
            "b": b.tolist(),
            "distance": dist
        }

# sort all pairs by distance, smallest first
sorted_distances = sorted(distances.items(), key=lambda x: x[1]["distance"])

# pick the top 10/1000 closest pairs
N = 1000  
selected_pairs = [pair for pair, info in sorted_distances[:N]]

# merge pairs into groups if they overlap
merged = []

for pair in selected_pairs:
    pair_set = set(pair)
    found = False
    for group in merged:
        # check if pair shares any point with group and if so, merge into group
        if pair_set & group:  
            group.update(pair_set)  
            found = True
            break
    # create new group if no overlap with existing group
    if not found:
        merged.append(pair_set)  

# repeat process for repeat merges until all connections found
changed = True
while changed:
    changed = False
    new_merged = []
    while merged:
        first = merged.pop(0)
        i = 0
        while i < len(merged):
            if first & merged[i]: 
                first.update(merged.pop(i))
                changed = True
            else:
                i += 1
        new_merged.append(first)
    merged = new_merged

# convert each set of indexes to a sorted list
merged = [sorted(list(g)) for g in merged]

# get coordinates for each group
groups_with_coords = []
for group in merged:
    group_coords = [data[idx] for idx in group]  
    groups_with_coords.append(group_coords)

junction_counts = []

# print each group and count junction boxes
for i, group in enumerate(groups_with_coords):
    #print(f"group {i+1}:")
    #for j, coords in enumerate(group):
        #print(f"  Point {j+1}: {coords}")
    junction_counts.append(len(group))
    #print(f"junction boxes: {len(group)}")

# find top 3 largest groups and multiply their sizes
top_3 = heapq.nlargest(3, junction_counts)
#print("top 3 circuit sizes:", top_3)
print("Product:", math.prod(top_3))

Product: 32103
CPU times: user 3.08 s, sys: 118 ms, total: 3.19 s
Wall time: 3.16 s


### Part B

In [5]:
%%time 

import numpy as np

# read input file and split each line by comma
with open("input8.txt") as file:
    data = [line.strip().split(',') for line in file]

# convert each line from strings to a tuple of ints
data = [tuple(map(int, coords)) for coords in data]

n = len(data)

# find distances between every pair of points
distances = {}
for i in range(n):
    a = np.array(data[i], dtype=int)
    for j in range(i+1, n):
        b = np.array(data[j], dtype=int)
        # euclidean distance
        dist = float(np.linalg.norm(a - b))  
        distances[(i, j)] = dist 

# sort all pairs by distance, smallest first
sorted_distances = sorted(distances.items(), key=lambda x: x[1])

# create initial groups, each point in its own set
merged = [{i} for i in range(n)]
last_pair = None

# create a mapping from each point to its group
point_to_group = {i: {i} for i in range(n)}
merged_groups = list(point_to_group.values())
last_pair = None

for (i, j), dist in sorted_distances:
    group_i = point_to_group[i]
    group_j = point_to_group[j]
    
    if group_i is not group_j:  # only merge if in different groups
        # merge group_j into group_i
        group_i.update(group_j)
        # update mapping for all points in group_j
        for p in group_j:
            point_to_group[p] = group_i
        # remove group_j from the merged_groups list
        merged_groups.remove(group_j)
        last_pair = (i, j)  # track last pair
        
    if len(merged_groups) == 1:  # stop when all points connected
        break

# multiply x coordinates of the last pair
x1, x2 = data[last_pair[0]][0], data[last_pair[1]][0]
print("last pair of coordinates:", data[last_pair[0]], data[last_pair[1]])
print("product of X coordinates:", x1 * x2)

last pair of coordinates: (87002, 11886, 67321) (93488, 21925, 60118)
product of X coordinates: 8133642976
CPU times: user 2.13 s, sys: 35.9 ms, total: 2.16 s
Wall time: 2.15 s
